# Compare NLP Techniques: Build Model On word2vec Vectors

### Read In Cleaned Text

In [1]:
# Load the cleaned training and test sets
import gensim
import numpy as np
import pandas as pd

X_train = pd.read_csv('../../../data/X_train.csv')
X_test = pd.read_csv('../../../data/X_test.csv')
y_train = pd.read_csv('../../../data/y_train.csv')
y_test = pd.read_csv('../../../data/y_test.csv')

### Create word2vec Vectors

In [3]:
# Train a basic word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [6]:
# Replace the words in each text message with the learned word vector
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_train['clean_text']])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words])
                         for ls in X_test['clean_text']])

f:\Shadman\advanced-nlp-with-python-for-machine-learning\venv\lib\site-packages\ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.
f:\Shadman\advanced-nlp-with-python-for-machine-learning\venv\lib\site-packages\ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  


In [7]:
# Average the word vectors for each sentence (and assign a vector of zeros if the model
# did not learn any of the words in the text message during training
X_train_vect_avg = []
for v in X_train_vect:
    if v.size:
        X_train_vect_avg.append(v.mean(axis=0))
    else:
        X_train_vect_avg.append(np.zeros(100, dtype=float))
        
X_test_vect_avg = []
for v in X_test_vect:
    if v.size:
        X_test_vect_avg.append(v.mean(axis=0))
    else:
        X_test_vect_avg.append(np.zeros(100, dtype=float))

In [8]:
# What does the unaveraged version look like?
X_train_vect[0]

array([[-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       [-0.00053623,  0.00023643,  0.00510335, ..., -0.00704156,
         0.00090146,  0.00639253],
       ...,
       [-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588],
       [-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588],
       [-0.00861969,  0.00366574,  0.00518988, ..., -0.00239151,
        -0.0095101 ,  0.00450588]], dtype=float32)

In [9]:
# What does the averaged version look like?
X_train_vect_avg[0]

array([-2.8457877e-03,  1.2162324e-03,  5.1280735e-03,  8.0757486e-03,
       -4.5115589e-03, -6.8456284e-03,  4.9293698e-03,  8.1370715e-03,
       -4.3938928e-03, -4.4519873e-03,  5.1545831e-03, -3.4864661e-03,
       -4.8404424e-03,  6.7113335e-03, -2.5136748e-03,  7.6732121e-04,
        3.9976272e-03,  2.8601217e-03, -7.0006279e-03, -6.9096717e-03,
        5.8936551e-03,  2.3304631e-03,  7.2237030e-03, -2.2717144e-03,
        6.4691044e-03, -1.5997135e-03, -2.0853826e-03,  5.3770351e-03,
       -5.8696200e-03, -8.9396565e-04, -2.5183163e-03, -1.9107286e-03,
        6.6417023e-03, -6.8553020e-03, -5.6674343e-04, -5.8792240e-04,
        7.7384762e-03, -2.4931848e-03,  2.7575425e-03, -7.4597716e-04,
       -4.6030884e-03,  1.5796365e-03, -8.8728089e-03, -3.2386624e-03,
       -9.1074052e-04,  2.0439599e-03, -3.7755780e-03,  6.4260559e-03,
        3.9903158e-03,  7.1065421e-03, -3.5934921e-03,  4.9408694e-04,
       -3.0137782e-03,  1.5801523e-03,  5.8021625e-03, -7.9247815e-04,
      

### Fit RandomForestClassifier On Top Of Word Vectors

In [10]:
# Instantiate and fit a basic Random Forest model on top of the vectors
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10)
rf_model = rf.fit(X_train_vect_avg, y_train.values.ravel())

In [11]:
# Use the trained model to make predictions on the test data
y_pred = rf_model.predict(X_test_vect_avg)

In [12]:
# Evaluate the predictions of the model on the holdout test set
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
print('Precision: {} / Recall: {} / Accuracy: {}'.format(
    round(precision, 3), round(recall, 3), round((y_pred==y_test['label']).sum()/len(y_pred), 3)))

Precision: 0.525 / Recall: 0.211 / Accuracy: 0.866
